## Этап 1. Сбор и подготовка данных

In [1]:
from src.data_utils import (
    prepare_data,
    create_data_loaders,
    save_artifacts
)

# запуск подготовки
train_texts, val_texts, test_texts, word_to_idx, idx_to_word, vocab_size, seq_len = prepare_data(
    file_path='data/raw_data.csv',
    sample_ratio=0.2,
    min_len=4,
    max_len=16,
    seq_len=10,
    random_seed=42
)

# даталоадеры
train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = create_data_loaders(
    train_texts, val_texts, test_texts,
    word_to_idx=word_to_idx,
    seq_len=seq_len,
    batch_size=128
)

# сохраняем
save_artifacts(word_to_idx, idx_to_word, vocab_size, seq_len, train_dataset, val_dataset, test_dataset)

загружаем и подготавливаем данные
----------------------------------------
загружено 1600498 строк из файла
всего строк в файле: 1600498
оставлено после сэмплирования: 320099 строк
очищено текстов: 320099
----------------------------------------

примеры очистки:
до:  @bkajino That awesome was for the walk, not the blisters
после: that awesome was for the walk not the blisters

----------------------------------------
до:  @trent_reznor @mariqueen it makes me sad to see all the bs ppl are writing about and to you two.  I don't know why they can't let you be..
после: it makes me sad to see all the bs ppl are writing about and to you two i don't know why they can't let you be

----------------------------------------
до:  i give up on them !
после: i give up on them

----------------------------------------
до:  @jeanettejoy  Well said.  Never follow someone because of their name, follow because they have something worth listening too
после: well said never follow someone because of thei

## Этап 2. Объявление модели

In [2]:
import pickle
import torch
from src.lstm_model import LSTMTokenizerModel

# словарь
with open('data/vocab_final.pkl', 'rb') as f:
    vocab_data = pickle.load(f)

# данные из словаря
word_to_idx = vocab_data['word_to_idx']
idx_to_word = vocab_data['idx_to_word']

# параметры модели
vocab_size = vocab_data['vocab_size']
embed_dim = 128
hidden_dim = 256
num_layers = 2
seq_len = vocab_data['seq_len'] 

# экземпляр модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMTokenizerModel(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers
).to(device)

print("-" * 40)
print(model)
print("-" * 40)
print(f"размер словаря: {vocab_size}")
print(f"модель создана на устройстве: {device}")
print(f"количество параметров: {sum(p.numel() for p in model.parameters()):,}")
print("-" * 40)

----------------------------------------
LSTMTokenizerModel(
  (embedding): Embedding(61013, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=61013, bias=True)
)
----------------------------------------
размер словаря: 61013
модель создана на устройстве: cuda
количество параметров: 24,411,605
----------------------------------------


## Этап 3. Тренировка модели

In [1]:
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from src.lstm_model import generate
from src.lstm_train import train 
from src.eval_lstm import evaluate_final

# запуск обучения
model, word_to_idx, idx_to_word, seq_len, device, test_loader = train()

# финальная оценка
evaluate_final(model, test_loader, word_to_idx, idx_to_word, seq_len, device)

# примеры генерации
print("-" * 40)
print("генерация")
print("-" * 40)
print('i ->', generate(model, 'i', word_to_idx, idx_to_word, seq_len, temperature=0.8, top_k=10))
print('i love ->', generate(model, 'i love', word_to_idx, idx_to_word, seq_len, temperature=0.8, top_k=10))
print('today ->', generate(model, 'today', word_to_idx, idx_to_word, seq_len, temperature=0.8, top_k=10))
print("-" * 40)

словарь загружен: 61013 токенов
пример: 'the' -> 22408
длина контекста (seq_len): 10
размер обучающей выборки: 1363329
пример входа: (tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0, 39991]), tensor(15758)) -> you
используем устройство: cuda
----------------------------------------

дебаг: проверка данных
----------------------------------------
пример x_batch[0]: [0, 0, 0, 0, 35873, 7055, 29026, 19935, 59687, 48108]
y_batch.min(): 42
y_batch.max(): 60123
forward и loss работают
----------------------------------------
фиксированный пример для сравнения каждую эпоху
----------------------------------------
контекст: thanks
реальное продолжение: for
ожидаем: thanks for
----------------------------------------


epoch 1: 100%|██████████| 10652/10652 [02:57<00:00, 59.94it/s]


epoch 1 | train loss: 74366.607 | val loss: 6.680 | val acc: 9.18% | val ppl: 796.33 | val rouge-l: 0.092
----------------------------------------
прогресс модели (сравнение с истиной)
----------------------------------------
предсказание: i
топ-3: ['i', 'for', 'to']
реальность:   for
----------------------------------------
epoch 1 | lr: 5.00e-05 | val loss: 6.680


epoch 2: 100%|██████████| 10652/10652 [02:58<00:00, 59.70it/s]


epoch 2 | train loss: 68396.235 | val loss: 6.425 | val acc: 10.87% | val ppl: 617.17 | val rouge-l: 0.109
----------------------------------------
прогресс модели (сравнение с истиной)
----------------------------------------
предсказание: for
топ-3: ['for', 'to', 'i']
реальность:   for
----------------------------------------
epoch 2 | lr: 5.00e-05 | val loss: 6.425


...

epoch 20 | train loss: 53576.233 | val loss: 5.947 | val acc: 16.15% | val ppl: 382.48 | val rouge-l: 0.162
----------------------------------------
прогресс модели (сравнение с истиной)
----------------------------------------
предсказание: for
топ-3: ['for', 'i', 'to']
реальность:   for
----------------------------------------
epoch 20 | lr: 2.50e-05 | val loss: 5.947


epoch 21: 100%|██████████| 10652/10652 [02:57<00:00, 59.93it/s]


epoch 21 | train loss: 53360.943 | val loss: 5.945 | val acc: 16.17% | val ppl: 381.78 | val rouge-l: 0.162
----------------------------------------
прогресс модели (сравнение с истиной)
----------------------------------------
предсказание: for
топ-3: ['for', 'i', 'to']
реальность:   for
----------------------------------------
epoch 21 | lr: 2.50e-05 | val loss: 5.945


epoch 22: 100%|██████████| 10652/10652 [02:58<00:00, 59.52it/s]


epoch 22 | train loss: 53168.690 | val loss: 5.948 | val acc: 16.23% | val ppl: 382.88 | val rouge-l: 0.163
----------------------------------------
прогресс модели (сравнение с истиной)
----------------------------------------
предсказание: for
топ-3: ['for', 'i', 'to']
реальность:   for
----------------------------------------
ранняя остановка на эпохе 22
загружена лучшая модель
----------------------------------------
дополнительные метрики
test loss: 5.928 | test acc: 15.88% | test ppl: 375.48 | test rouge-l: 0.159
----------------------------------------
----------------------------------------

сгенерированные примеры
----------------------------------------
→ i m so glad to see you in the sun is shining and i m back
→ i love u when that was a great show i m sorry i love you too much
→ today is an awesome day off to work tomorrow no one is coming here soon to
→ life is going to be awesome and i feel like crap is in the hospital now
→ i can't see my picture in the morning i don't 

## Этап 4. Использование предобученного трансформера distilgpt2

In [6]:
from src.eval_transformer_pipeline import TransformerModel

# экземпляр модели
model = TransformerModel()

# данные
test_texts, val_texts = model.prepare_data('data/raw_data.csv')

# валидация модели
avg_rouge, examples = model.validate_model(val_texts)

# генерируем тексты
generation_results = model.generate_texts(test_texts)

загружаем модель distilgpt2
используется GPU
загружаем и подготавливаем данные
загружено 1600498 твитов
очищено текстов: 1600498
отфильтровано текстов: 1251724
валидационная выборка: 150 текстов
----------------------------------------
валидация модели
----------------------------------------
выполняем оценку модели на валидационной выборке

----------------------------------------
результаты валидации
----------------------------------------
метрика: ROUGE-L (точность предсказания следующего слова)
количество оценок: 99
средний ROUGE-L: 0.0976 ± 0.2930
медиана ROUGE-L: 0.0000

лучшие примеры предсказаний:
----------------------------------------
пример 1:
контекст:    'what tragedy and disaster in'
истинное:    'the'
предсказанное: 'the'
ROUGE-L:     1.0000

пример 2:
контекст:    'i miss him can't wait'
истинное:    'to'
предсказанное: 'to'
ROUGE-L:     1.0000

пример 3:
контекст:    'has lost his ring it'
истинное:    's'
предсказанное: '’s'
ROUGE-L:     1.0000

пример 4:
контекст: 

## Этап 5. Формулирование выводов


> **Выводы:** Не могу дать однозначный ответ, что лучше. 

##### Начнём с метрик
- Основная метрика по ТЗ, ROUGE, практически бесполезна для оценки distilgpt2, так как эта модель генерирует принципиально новые тексты, а не копирует фразы из эталона (и мы буквально берём из коробки готовый пайплайн, не проводя дообучение на своих данных). 
Для LSTM, которая часто работает ближе к шаблонному предсказанию, ROUGE ещё может иметь некоторый смысл.
- Что касается других метрик, таких как accuracy или perplexity, их использование также некорректно для сравнения, так как мы опять же берём "из коробки" distilgpt2, нам нам ни логиты, ни вероятности не дадут 
(Измерить вроде можно, но скорее бессмысленно (метрики accuracy и perplexity предназначены для оценки языковых моделей на задачах предсказания следующего слова и плохо подходят для оценки качества генерации текста), чем сложно + вручную уже запускала distilgpt2, слишком запарно для данной задачи)

##### Основной акцент в сравнении приходится делать на анализ примеров генерации и технические характеристики
Поэтому, выбираем:
- LSTM, если приоритетом является вычислительная эффективность модели, а не качество текста (жёсткие аппаратные ограничения, например, чат-бот с очень узкой тематикой, мне представляется мобильная игра, в которой чат-бот должен следовать только своему лору (LOR'у, ограниченному набору правил и контекста) и словарь не особо большой).
- distilgpt2, если приоритетом является связность (человекочитаемость) и разнообразие генерируемого текста (диалоговые системы, автоматическое дополнение текста, вывод инструкций по узкой тематике, чат-бот для работников компании, не крупной, при больших данных те же x5 используют не лёгкие модели, а берут API OpenAI, например).


In [2]:

import torch
import pickle
from src.lstm_model import LSTMTokenizerModel, generate
from src.eval_transformer_pipeline import TransformerModel

# загрузка lstm
print("загружаем LSTM модель")
with open('data/vocab_final.pkl', 'rb') as f:
    vocab_data = pickle.load(f)

word_to_idx = vocab_data['word_to_idx']
idx_to_word = vocab_data['idx_to_word']
seq_len = vocab_data['seq_len']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lstm_model = LSTMTokenizerModel(len(word_to_idx)).to(device)
lstm_model.load_state_dict(torch.load('models/best_lstm_model_final.pt', map_location=device))
lstm_model.eval()

# загрузка distilgpt2 модели
print("-" * 40)
print("загружаем distilgpt2 модель")
gpt2_model = TransformerModel()
gpt2_generator = gpt2_model.generator

# 10 примеров для генерации
test_examples = [
    "i love to play with my",
    "today is a beautiful day for",
    "t can't believe how fast time",
    "you should try this amazing",
    "we are going to the",
    "the weather is really nice",
    "i think we should go to",
    "this is the best movie I",
    "she always knows how to make",
    "life is too short to"
]

print("-" * 40)
print("сравнение генераций")
print("-" * 40)

# генерация для тестовых примеров
for i, prompt in enumerate(test_examples, 1):
    print(f"\nпример {i}:")
    print(f"промпт: '{prompt}'")
    print("-" * 40)
    
    # генерация lstm
    lstm_output = generate(lstm_model, prompt, word_to_idx, idx_to_word, seq_len, max_len=20)
    print(f"lstm:      {lstm_output}")
    
    # генерация distilgpt2
    try:
        gpt2_result = gpt2_generator(
            prompt,
            max_new_tokens=20,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=gpt2_generator.tokenizer.eos_token_id
        )
        gpt2_output = gpt2_result[0]['generated_text'].strip()
        print(f"distilgpt2: {gpt2_output}")
    except Exception as e:
        print(f"distilgpt2: ошибка генерации - {e}")
    
    print("-" * 40)

# интерактивный ввод промпта
print("-" * 40)
print("интерактивная генерация")
print("-" * 40)

while True:
    user_prompt = input("\nвведите свой промпт (или 'exit' для выхода): ")
    
    if user_prompt.lower() == 'exit':
        break
    
    if not user_prompt.strip():
        continue
    
    print(f"\nпромпт: '{user_prompt}'")
    print("-" * 40)
    
    # генерация lstm
    try:
        lstm_output = generate(lstm_model, user_prompt, word_to_idx, idx_to_word, seq_len, max_len=20)
        print(f"lstm:      {lstm_output}")
    except Exception as e:
        print(f"lstm: ошибка генерации - {e}")
    
    # генерация distilgpt2
    try:
        gpt2_result = gpt2_generator(
            user_prompt,
            max_new_tokens=30,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=gpt2_generator.tokenizer.eos_token_id
        )
        gpt2_output = gpt2_result[0]['generated_text'].strip()
        print(f"distilgpt2: {gpt2_output}")
    except Exception as e:
        print(f"distilgpt2: ошибка генерации - {e}")
    
    print("-" * 40)

print("\nзавершено")


загружаем LSTM модель
----------------------------------------
загружаем distilgpt2 модель
загружаем модель distilgpt2
используется GPU
----------------------------------------
сравнение генераций
----------------------------------------

пример 1:
промпт: 'i love to play with my'
----------------------------------------
lstm:      i love to play with my favorite babies and comment when i have anything jet meka if no legal either is
distilgpt2: i love to play with my friends.


If you would like to see a video for my next book, please visit
----------------------------------------

пример 2:
промпт: 'today is a beautiful day for'
----------------------------------------
lstm:      today is a beautiful day for it hahaha sits it sucks i have went on what to do fresh address long
distilgpt2: today is a beautiful day for all of us, and for all of you.”


In addition to the holiday
----------------------------------------

пример 3:
промпт: 't can't believe how fast time'
------------------